In [1]:
import pandas as pd
import numpy as np
import faiss

In [46]:
# View Config
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 400)

In [3]:
index = faiss.read_index('index')

In [5]:
df = pd.read_csv('netflix_titles.csv')

In [7]:
def create_textual_representation(row):
    textual_representation = f"""Type: {row['type']},
    Title: {row['title']},
    Director: {row['director']},
    Cast: {row['cast']},
    Release Year: {row['release_year']},
    Release Year: {row['listed_in']},
    Description: {row['description']},
    """

    return textual_representation

In [9]:
df['textual_representation'] = df.apply(create_textual_representation, axis=1)

In [48]:
df[df.title.str.contains('Shutter')]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,textual_representation
1358,s1359,Movie,Shutter Island,Martin Scorsese,"Leonardo DiCaprio, Mark Ruffalo, Ben Kingsley, Max von Sydow, Michelle Williams, Emily Mortimer, Patricia Clarkson, Jackie Earle Haley, Ted Levine, John Carroll Lynch, Elias Koteas",United States,"February 1, 2021",2010,R,139 min,Thrillers,A U.S. marshal's troubling visions compromise his investigation into the disappearance of a patient from a hospital for the criminally insane.,"Type: Movie,\n Title: Shutter Island,\n Director: Martin Scorsese,\n Cast: Leonardo DiCaprio, Mark Ruffalo, Ben Kingsley, Max von Sydow, Michelle Williams, Emily Mortimer, Patricia Clarkson, Jackie Earle Haley, Ted Levine, John Carroll Lynch, Elias Koteas,\n Release Year: 2010,\n Release Year: Thrillers,\n Description: A U.S. marshal's troubling visions compromise his investi..."
8013,s8014,Movie,Shutter,"Banjong Pisanthanakun, Parkpoom Wongpoom","Ananda Everingham, Natthaweeranuch Thongmee, Achita Sikamana, Unnop Chanpaibool, Titikarn Tongprasearth, Sivagorn Muttamara, Chachchaya Chalemphol, Kachormsak Naruepatr",Thailand,"September 5, 2018",2004,TV-MA,96 min,"Horror Movies, International Movies","After killing a young girl in a hit-and-run accident, a couple is haunted by more than just the memory of their deadly choice.","Type: Movie,\n Title: Shutter,\n Director: Banjong Pisanthanakun, Parkpoom Wongpoom,\n Cast: Ananda Everingham, Natthaweeranuch Thongmee, Achita Sikamana, Unnop Chanpaibool, Titikarn Tongprasearth, Sivagorn Muttamara, Chachchaya Chalemphol, Kachormsak Naruepatr,\n Release Year: 2004,\n Release Year: Horror Movies, International Movies,\n Description: After killing a young gir..."


In [39]:
fav = df.iloc[1358]

In [41]:
from pymilvus import model

embedding_fn = model.DefaultEmbeddingFunction()

In [50]:
print(fav['textual_representation'])

Type: Movie,
    Title: Shutter Island,
    Director: Martin Scorsese,
    Cast: Leonardo DiCaprio, Mark Ruffalo, Ben Kingsley, Max von Sydow, Michelle Williams, Emily Mortimer, Patricia Clarkson, Jackie Earle Haley, Ted Levine, John Carroll Lynch, Elias Koteas,
    Release Year: 2010,
    Release Year: Thrillers,
    Description: A U.S. marshal's troubling visions compromise his investigation into the disappearance of a patient from a hospital for the criminally insane.,
    


In [58]:
query_vectors = embedding_fn.encode_queries([fav['textual_representation']])

In [60]:
embedding = np.array(query_vectors, dtype='float32')

In [62]:
# Return top 5 most similar results
D, I = index.search(embedding, 5)  # D - Distance, I - Index

In [74]:
best_matches = np.array(df['textual_representation'])[I.flatten()]

In [76]:
for i in best_matches:
    print('Next Movie:')
    print(i)
    print()

Next Movie:
Type: Movie,
    Title: Shutter Island,
    Director: Martin Scorsese,
    Cast: Leonardo DiCaprio, Mark Ruffalo, Ben Kingsley, Max von Sydow, Michelle Williams, Emily Mortimer, Patricia Clarkson, Jackie Earle Haley, Ted Levine, John Carroll Lynch, Elias Koteas,
    Release Year: 2010,
    Release Year: Thrillers,
    Description: A U.S. marshal's troubling visions compromise his investigation into the disappearance of a patient from a hospital for the criminally insane.,
    

Next Movie:
Type: Movie,
    Title: Hugo,
    Director: Martin Scorsese,
    Cast: Ben Kingsley, Sacha Baron Cohen, Asa Butterfield, Chloë Grace Moretz, Ray Winstone, Emily Mortimer, Christopher Lee, Helen McCrory, Michael Stuhlbarg, Frances de la Tour, Jude Law, Johnny Depp, Edmund Kingsley, Richard Griffiths, Angus Barnett, Michael Pitt, Gulliver McGrath, Eric Moreau, Chloe Grace Moretz,
    Release Year: 2011,
    Release Year: Children & Family Movies, Dramas,
    Description: Living inside a Par